# Features

1. Vector storage
2. Retriver
3. Chat engine
4. Memory system
5. Conversational query 
6. Conversational query with memory

In [24]:
import pandas as pd
from datetime import datetime

from IPython.display import display
from IPython.display import Markdown

import google.generativeai as genai

from utils import read_config, OracleAgent

from llama_index.core.schema import Document
from llama_index.core import VectorStoreIndex, StorageContext, Settings
from llama_index.vector_stores.chroma import ChromaVectorStore
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

import chromadb

In [17]:
info_path = '.env/connections.json'
info = read_config(info_path)

oracle_agent = OracleAgent(info['DW_conn_info'])

API_KEY= info['gkey']
genai.configure(api_key=API_KEY)
model = genai.GenerativeModel('gemini-1.5-flash')


ProgrammingError: Oracle Client library has already been initialized

In [18]:
query = """
select rowid, dashboard_name, report_name, page_name, pic_name, llm, llm_question 
from bi_dashboard
"""

df_dashboard = oracle_agent.read_table(query=query)
df_dashboard.to_csv('opt/dashboard_data.csv', index=False, encoding='utf-16')

DatabaseError: (cx_Oracle.DatabaseError) ORA-12541: Cannot connect. No listener at host 10.25.66.64 port 1521.
Help: https://docs.oracle.com/error-help/db/ora-12541/
(Background on this error at: https://sqlalche.me/e/20/4xp6)

In [2]:
df_dashboard = pd.read_csv('files/dashboard_data.csv', encoding='utf-16')

In [3]:
df_dashboard = df_dashboard.rename(columns={"llm": "description"})

In [4]:
df_dashboard.head()

,rowid,dashboard_name,report_name,page_name,pic_name,description,llm_question
0,AAi0soABZAAAACLAAC,投控經營分析,華紙-營運績效分析月報,紙機生產狀況,紙機生產狀況-得率趨勢圖.jpg,這張圖表顯示的是 2024 年 1 月到 5 月的月度效率數據。圖表上的數字代表各個月的效率...,這張圖片顯示了華紙在2024年1月到5月的紙機生產狀況，以下列出10個關於這張圖片的問題：\...
1,AAi0soABZAAAACLAAF,投控經營分析,華紙-營運績效分析月報,號機直接法損益,號機直接法損益-完整畫面.jpg,這張圖片包含了一份營運績效分析月報，其中包含以下指標：\n\n**成本**:\n* 銷貨成本...,這張圖片顯示了華紙的營運績效分析月報，可以回答以下問題：\n\n1. **華紙的總成本是多少...
2,AAi0soABZAAAACOAAB,投控經營分析,華紙-營運績效分析日報,工廠面,工廠面3.JPG,這張圖片包含報表，包含投控經營分析華紙-營運績效分析日報工廠面的一些相關分析指標。\n\n*...,這張圖片顯示的是工廠生產相關的數據，可以提出以下問題：\n\n1. 今日生產的紙張總量是多...
3,AAi0soABZAAAACMAAE,投控經營分析,華紙-營運績效分析月報,銷售面,銷售面-事故金額狀況.jpg,這張圖片顯示了**華紙-營運績效分析月報銷售面**的數據，以柱狀圖的形式呈現。 \n\n**...,這張圖片顯示的是華紙-營運績效分析月報銷售面的數據，可以回答以下問題：\n\n1. 2024...
4,AAi0soABZAAAACUAAA,投控經營分析,華紙-營運績效分析月報,漿線生產狀況,漿線生產狀況-停機時間.jpg,"這張圖片只顯示了一項指標：**停機時間**，單位為分鐘，數值为50,750。\n\n如果需要...",這張圖片只顯示一個標題「停機時間（Min）」和一個單純的藍色圓形，以及下方顯示「非計劃停機 ...


In [13]:
model_name = "sentence-transformers/distiluse-base-multilingual-cased-v1"
embeddings = HuggingFaceEmbedding(model_name=model_name)

e:\miniconda3\envs\bibot\lib\site-packages\huggingface_hub\file_download.py:157: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Sean\AppData\Local\llama_index\models--sentence-transformers--distiluse-base-multilingual-cased-v1. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


In [21]:
chroma_client = chromadb.PersistentClient(path="./chroma_db")
chroma_collection = chroma_client.get_or_create_collection("DashboardQA")

In [17]:
def df_to_documents(df):
    return [Document(text=row['description'], metadata={col: row[col] for col in df.columns if col != 'descriptoin'}) for _, row in df.iterrows()]

In [18]:
documents = df_to_documents(df_dashboard)

In [22]:
documents[:2]

[Document(id_='faed5077-e92e-4475-bebf-46b8fa80e851', embedding=None, metadata={'rowid': 'AAi0soABZAAAACLAAC', 'dashboard_name': '投控經營分析', 'report_name': '華紙-營運績效分析月報', 'page_name': '紙機生產狀況', 'pic_name': '紙機生產狀況-得率趨勢圖.jpg', 'description': '這張圖表顯示的是 2024 年 1 月到 5 月的月度效率數據。圖表上的數字代表各個月的效率百分比。 \n\n您可以查詢：\n\n- **效率**（以百分比顯示）： 圖表顯示了每個月的效率，從 1 月到 5 月。\n- **月份**： 圖表顯示了每個月的效率，從 1 月到 5 月。\n- **2024 年**： 圖表顯示了 2024 年的數據。\n\n圖表中缺少 6 月份的數據。', 'llm_question': '這張圖片顯示了華紙在2024年1月到5月的紙機生產狀況，以下列出10個關於這張圖片的問題：\n\n1. 圖表顯示的是哪個月的紙機生產狀況？\n2. 圖表的縱軸代表什麼？\n3. 1月份的紙機生產狀況如何？\n4. 2月份的紙機生產狀況如何？\n5. 3月份的紙機生產狀況如何？\n6. 4月份的紙機生產狀況如何？\n7. 5月份的紙機生產狀況如何？\n8. 6月份的紙機生產狀況如何？\n9. 2024年1月到5月之間，哪個月的紙機生產狀況最好？\n10. 2024年1月到5月之間，哪個月的紙機生產狀況最差？\n'}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, text='這張圖表顯示的是 2024 年 1 月到 5 月的月度效率數據。圖表上的數字代表各個月的效率百分比。 \n\n您可以查詢：\n\n- **效率**（以百分比顯示）： 圖表顯示了每個月的效率，從 1 月到 5 月。\n- **月份**： 圖表顯示了每個月的效率，從 1 月到 5 月。\n- **2024 年**： 圖表顯示了 2024 年的數據。\n\n圖表中缺少 6 月份的數據。', mimety

In [28]:
Settings.chunk_size = 2048
Settings.chunk_overlap = 50
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
storage_context = StorageContext.from_defaults(vector_store=vector_store)
vector_store_index = VectorStoreIndex.from_documents(
    documents=documents,
    storage_context=storage_context,
    embed_model=embeddings
)